<br>
<a href="https://www.nvidia.cn/training/online/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用 NVIDIA Earth-2 部署 AI 天气模型</h1>
<h2><b>练习 1：</b> 预测</h2>
<br>

在课程的第一部分, 我们将使用全球 AI 气象模型创建中期天气预报。天气对各行各业有着显著的影响。例如，极端天气事件会扰乱供应链、能源基础设施，并对公众构成风险。此外，天气条件会影响从农业产量到能源消耗和生产的方方面面。准确的天气预报使利益相关者能够做出明智的决策、管理风险并优化他们的策略。通过理解天气模式，企业可以更好地预测市场波动，保护他们的投资。

全球 AI 气象模型根据当前的气候状态预测大气的未来状态。与数值天气模型相比，AI 模型生成天气预报的速度快了几个数量级，并且运行在更小的硬件上。这些模型在大型历史气象数据集上进行训练，最著名的是 ERA5，欧洲中期天气预报中心大气重分析的第五代。推理通过自回归回放进行，这意味着之前时间步的模型输出会被反馈到模型中，以计算下一个时间步。

中期天气预报可以覆盖未来的 10-14 天，并且这被认为是可能进行确定性预测的时间范围。确定性预测的目标是预测天气随时间变化的单一最可能演变。然而，在构建可靠的天气应用程序时，我们一般关注概率预测。概率预测能告诉我们某些天气事件或天气条件发生的可能性。虽然数值概率预测计算量极大，但 AI 模型可以快速生成概率预测，使其在定制应用中非常高效和可访问。

In [1]:
import numpy as np

from earth2studio import run
from earth2studio.data import GFS, NCAR_ERA5
from earth2studio.io import KVBackend
from earth2studio.lexicon import NCAR_ERA5Lexicon
from earth2studio.models.px import SFNO
from earth2studio.perturbation import CorrelatedSphericalGaussian, SphericalGaussian 
from earth2studio.utils.time import to_time_array
from hens import HemisphericCentredBredVector, get_noise_vector
from plot import animate_local, figure_global, plot_global_ensemble, plot_perturbations
from utils import get_lat_lon, get_locations

%matplotlib inline

## 示例

下面就开始创建接下来几天的确定性天气预报。这将演示如何轻松地使用 [Earth2Studio](https://github.com/NVIDIA/earth2studio) 运行自定义天气预报。之后，我们会深入细节。目前，我们只需四行代码来定义模型、数据源、IO 后端，最后启动工作流。请将 `"2025-04-01"` 改为今天的日期并执行下面的单元格（数据每天的 UTC 时间大约在早上 6 点可用，如果今天的数据还没有，请改为昨天的日期）。

In [2]:
model = SFNO.load_model(SFNO.load_default_package())
gfs = GFS()
io = KVBackend()
io = run.deterministic(["2025-04-01"], 16, model, gfs, io)

2025-07-06 07:57:45.034 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-07-06 07:57:45.172 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda


Fetching GFS for 2025-04-01 00:00:00: 100%|██████████| 73/73 [00:02<00:00, 29.71it/s]


2025-07-06 07:57:48.161 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from GFS
2025-07-06 07:57:48.374 | INFO     | earth2studio.run:deterministic:136 - Inference starting!


Running inference:   0%|          | 0/17 [00:00<?, ?it/s]

2025-07-06 07:57:54.678 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete


希望天气晴朗！下面的动画显示了东地中海 2 米高度的温度 (`t2m`)。我们已经创建了全球预报，所以可以随意更改纬度 (`loc_lat`) 和经度 (`loc_lon`) 到您感兴趣的地方。此外，预报还包含多个变量，如风速（例如，`u10m`，`v10m`）和湿度（例如，`q1000`），您也可以进行探索。

In [3]:
ds = io.to_xarray()
animate_local(ds.t2m, ds.time[0], loc_lat=38.0, loc_lon=23.75, cb_label="Temperature [K]")

Creating animation.....................

## 数据源

为了理解发生了什么，先熟悉一些数据源和数据处理工具。地球观测数据复杂且可能非常庞大，因此使用有效的工具很重要。Earth2Studio 提供了一个统一的接口来访问不同的数据存储库，这让我们的工作更轻松。我们的第一步将是探索通过 NSF NCAR 镜像提供的 ECMWF ERA5 数据。ERA5 数据最初通过欧盟的 [Copernicus Climate Change Service](https://climate.copernicus.eu/) 提供。让我们开始创建我们的数据源。`NCAR_ERA5` 数据源可访问来自 NCAR 镜像的 ERA5 数据。

In [4]:
ncar_era5 = NCAR_ERA5(verbose=True)

因为不同的数据集和数据提供者使用不同的标识符来表示相同的变量，Earth2Studio 提供了词汇表来在这些标识符之间进行映射。我们可以查看 `NCAR_ERA5Lexicon` 来了解通过 `NCAR_ERA5` 提供哪些变量。Earth2Studio 保留这些词汇表，您可以在不同数据源中使用相同的变量名称。在多个不同压力水平上可用的变量命名规则通常是一个字母后面跟着压力水平（例如，`z500`，`q850`）。而单一水平上的变量通常遵循相同的命名规则（例如，`t2m`，`u10m`），但也可能有所偏离（例如，`tcwv`）。这些变量名一般与 [ECMWF 参数数据库](https://codes.ecmwf.int/grib/param-db/) 中的简称一致，尽管有些例外。

In [5]:
lexicon = NCAR_ERA5Lexicon()
print(list(lexicon.VOCAB))  # list of ERA5 variables

['z1', 'z2', 'z3', 'z5', 'z7', 'z10', 'z20', 'z30', 'z50', 'z70', 'z100', 'z125', 'z150', 'z175', 'z200', 'z225', 'z250', 'z300', 'z350', 'z400', 'z450', 'z500', 'z550', 'z600', 'z650', 'z700', 'z750', 'z775', 'z800', 'z825', 'z850', 'z875', 'z900', 'z925', 'z950', 'z975', 'z1000', 't1', 't2', 't3', 't5', 't7', 't10', 't20', 't30', 't50', 't70', 't100', 't125', 't150', 't175', 't200', 't225', 't250', 't300', 't350', 't400', 't450', 't500', 't550', 't600', 't650', 't700', 't750', 't775', 't800', 't825', 't850', 't875', 't900', 't925', 't950', 't975', 't1000', 'u1', 'u2', 'u3', 'u5', 'u7', 'u10', 'u20', 'u30', 'u50', 'u70', 'u100', 'u125', 'u150', 'u175', 'u200', 'u225', 'u250', 'u300', 'u350', 'u400', 'u450', 'u500', 'u550', 'u600', 'u650', 'u700', 'u750', 'u775', 'u800', 'u825', 'u850', 'u875', 'u900', 'u925', 'u950', 'u975', 'u1000', 'v1', 'v2', 'v3', 'v5', 'v7', 'v10', 'v20', 'v30', 'v50', 'v70', 'v100', 'v125', 'v150', 'v175', 'v200', 'v225', 'v250', 'v300', 'v350', 'v400', 'v450', 

可以通过告诉数据源我们感兴趣的时间和变量来检索数据。下面选择了2023年3月和2024年6月的两个日期，以及四个变量：2 米高度的温度（`t2m`），总大气柱水汽（`tcwv`），以及10米风速的 u 和 v 组件（`u10m`，`v10m`）。Earth2Studio 处理与远程服务器的所有通信，并保持一个本地缓存。数据源返回一个 xarray 的 `DataArray`，我们可以方便地将其转换为 xarray 的 `Dataset`。

In [6]:
times = to_time_array(["2023-03-24 12:00:00", "2024-06-17 18:00:00"])
variables = ["t2m", "tcwv", "u10m", "v10m"]

ds_ncar_era5 = ncar_era5(times, variables).to_dataset("variable")

Fetching ERA5: 100%|██████████| 8/8 [00:02<00:00,  2.67it/s]


这个 `Dataset` 包含两个时间步和所有四个变量的数据。我们来绘制总大气柱水汽（`tcwv`），它显示出一些不错的漩涡。课程中提供的 `plot.py` 文件包含一些预定义的绘图功能，可以更有趣地探索将要处理的数据。图中的海岸线是由 [cartopy](https://scitools.org.uk/cartopy/docs/latest/) 提供的，这是一个流行的地理空间可视化库。绘图功能还允许您选择中心纬度和经度的视角。您可以使用 `utils.py` 中的 `get_lat_lon` 来实现，它提供了全球一些城市的坐标（使用 `get_locations` 查看具体城市）。当然，您也可以选择自己的坐标。

In [ ]:
figure_global(ds_ncar_era5.isel(time=0).tcwv, *get_lat_lon("Cape Town"))

接下来，我们将从 [GFS 分析存储库](https://registry.opendata.aws/noaa-gfs-bdp-pds/)加载相同时间和变量的数据，这个存储库由 NOAA/NCEP 通过 AWS 开放数据注册中心提供。Earth2Studio 提供了一致的接口和变量映射，所以我们只需将数据源更改为 `GFS`。

In [ ]:
gfs = GFS()

ds_gfs = gfs(times, variables).to_dataset("variable")  # same as above

乍一看，数据看起来相似，但当您比较细节时，会发现一些差异。GFS 分析存储库在数据可用后不久就会更新，而 ERA5 重分析的更新滞后大约为 5 天。这使得 GFS 更适合实时预报。另一方面，ERA5 提供了一致的数据，跨越几个月，适合用于训练全球的 AI 气象模型和分析过去的天气事件。例如，在历史数据上运行预报（也称为向后预报）可以生成物理上可信的反事实热带气旋事件集，用于校准风险模型。

In [ ]:
figure_global(ds_gfs.isel(time=0).tcwv, *get_lat_lon("Cape Town"))

## 模型

我们的预报将由 FourCastNet 执行，这是一个基于球面傅里叶神经算子的全球 AI 天气模型（SFNO）。这个模型架构允许学习长期空间依赖关系，并利用地球等球面系统的对称性。

以下图形来自 [Li et al., 2021](https://arxiv.org/abs/2010.08895)，展示了傅里叶神经算子的结构。输入 `a(x)` 被投影到潜在空间表示（通过 `P`），经过多个傅里叶层处理，再通过 `Q` 投影回去以获得输出 `u(x)`。想了解更多，可以看看关于 [FNO](https://arxiv.org/abs/2010.08895) 和 [SFNO](https://arxiv.org/abs/2306.03838) 的出版物。

![FNO 结构](./images/fno.jpg "FNO architecture")

Earth2Studio 提供的预训练模型以包的形式存在，包含模型权重和额外参数，通过 `load_default_package` 加载。然后可以通过 `load_model` 加载这些包。您还可以使用 [NVIDIA PhysicsNeMo](https://github.com/NVIDIA/physicsnemo) 来训练自定义模型，它专注于实现物理-机器学习模型的规模化训练。

In [ ]:
model = SFNO.load_model(SFNO.load_default_package())

模型通过坐标描述它们的输入和输出接口。每个坐标对应输入和输出张量的一个维度。第一个维度是批次维度。接下来的维度对应于起始时间、预报中的前瞻时间、变量，最后是地理坐标。FourCastNet（SFNO）的预训练版本处理 73 个大气变量。因为输入和输出变量是一样的，所以我们可以将每个输出反馈回模型，以生成下一个时间步。时间步之间的间隔为 6 小时。

In [ ]:
print("Input coordinates: ", list(model.input_coords()))
print("Input variables:   ", list(model.input_coords()["variable"]))
print()
print("Output coordinates:", list(model.output_coords(model.input_coords())))
print("Output variables:  ", list(model.output_coords(model.input_coords())["variable"]))

## 干扰

天气预报面临两个主要的不确定性来源：测量和模型。气象测量来自许多不同的仪器，如船、浮标、飞机、卫星、气象站、气象气球等等。 有些测量的准确性较低，有些则较高，但每个测量总会存在至少微小的误差。这些不完美的数据随后通过一个与现实紧密相关但并不完美的模型进行同化。用于分析的预报模型也是如此。例如，小尺度过程只能通过近似参数化来建模。为了捕捉这些不确定性，我们需要运行一个概率预报，而为此设立的方法是集成预测。

要创建集成预测，我们多次运行相同时间范围的相同仿真，但设置不同的条件。至少，这意味着每次预报运行使用不同的扰动初始条件。通过这种方式，我们考虑了测量和同化的不确定性。此外，我们还可以通过改变模型权重或使用一个本质上是概率性的模型（例如基于扩散）来估计预测模型的不确定性。本课程将坚持使用初始条件扰动，保持权重固定。为了科学上更严格的设置，我们推荐来自 LBNL 和 NVIDIA 的研究人员发布的有关生成大型集成的两部分论文（[第一部分](https://arxiv.org/abs/2408.03100)和[第二部分](https://arxiv.org/abs/2408.01581v1)）。

Earth2Studio 提供了几种干扰方法。例如，`SphericalGaussian` 和 `Brown` 干扰向输入添加空间相关的噪声。这两种方法都接受一个参数来确定噪声的幅度以及额外的参数来定义噪声的形状。下面的图表展示了不同的 `SphericalGaussian` 的 `alpha` 和 `Brown` 的 `reddening` 参数值，较高的值使噪声更平滑。选择合适的干扰方法和参数取决于应用场景，需要根据课程第二部分讨论的内容进行验证。

In [ ]:
plot_perturbations([1.0, 1.5, 2., 2.5], lats=121, lons=240)

对于我们的集成预报，通过 `SphericalGaussian` 应用干扰，噪声幅度与每个变量的平均值（`model.center`）成正比。

In [ ]:
sg = SphericalGaussian(noise_amplitude=0.03 * model.center[None, None].to("cuda"))

## 推理

天气预报产生了大量数据，因此高效的 IO 处理非常重要。Earth2Studio 提供了不同文件类型的 IO 后端，并跟踪变量、维度和元数据。我们将使用一个内存中的键值存储，等到预报完成后，会把这些数据写入一个 NetCDF4 存档。之后就可以用自己喜欢的工具加载和分析这些数据，比如 xarray。

In [ ]:
io = KVBackend()

现在是为第一次集成预测定义一些参数的时候了。我们希望在 2024 年 7 月 2 日（`start_time`）开始预报，这个时间点正好在飓风 Beryl 降落在尤卡坦半岛和美国南部海岸之前。预报将生成 16 步（`nsteps`），每次间隔 6 小时，因此会持续 4 天。我们将从 16 个不同的扰动初始条件（`nensemble`）开始预报。FourCastNet (SFNO) 生成总共 73 个变量，包括温度、风速、位势高度、湿度及其他气象因素。您的应用可能只依赖于这些变量中的某些，因此 Earth2Studio 允许您通过 `output_coords` 参数指定一组输出变量。只有在这个列表中的变量会被保存到磁盘，从而显著减少输出的大小。记住，对于每个时间切片，721x1440 的经纬度数据点，一个未压缩的 32 位变量大约占用 4 MB 的磁盘空间。随着变量、时间跨度和集成复制的增加，这个空间需求会迅速累积。同样的 `output_coords` 参数也可以用来仅保存部分经纬度网格，例如。

我们会请求 `t2m`（2 米温度）、`u10m`（风的 u 分量，从西方来的风，10 米高度）、`u100m`（100 米高度的风的 u 分量）、`v10m`（风的 v 分量，来自南方的风，10 米高度）、`v100m`（100 米高度的风的 v 分量）、`tcwv`（总大气柱水汽）、`z500`（500 hPa 的位势高度）。为明确起见，预报所有 73 个变量都会生成，因为它们对于后续的预报步骤是需要的。但最后只有这 7 个变量会被存储到磁盘上。

In [ ]:
start_time = np.datetime64("2024-07-04 00:00:00")
nsteps = 32
nensemble = 16
batch_size = 16
output_coords = {"variable": np.array(["t2m", "u10m", "v10m", "u100m", "v100m", "tcwv", "z500"])}

Earth2Studio 提供了预定义的工作流，便于运行集成预报和其他用例。我们只需传递上述定义的参数。这个工作流会获取初始条件，设置 IO 存储，应用扰动，最后运行预报。

In [ ]:
io = run.ensemble(
    [start_time],
    nsteps=nsteps,
    nensemble=nensemble,
    prognostic=model,
    data=ncar_era5,
    io=io,
    perturbation=sg,
    batch_size=batch_size,
    output_coords=output_coords,
)

可以使用 xarray 探索输出数据，它提供了一些方便的数据处理功能。

In [ ]:
!mkdir -p /workspace/exercises/output && rm -f /workspace/exercises/output/exercise_01.nc
ds = io.to_xarray()
ds.to_netcdf("/workspace/exercises/output/exercise_01.nc")  # save for exercise 2

In [ ]:
ds

生成的数据集有五个坐标：

1. `ensemble`，每个 16 个集成成员都有一个预报
2. `time`，我们的开始时间
3. `lead_time`，包含由初始条件和 32 步预报组成的 33 个值
4. `lat`，721 个纬度值
5. `lon`，1440 个经度值

如前所述，数据集包含七个变量：`t2m`，`u10m`，`v10m`，`u100m`，`v100m`，`tcwv`，和 `z500`。课程的这一部分以简单的图示结束，接下来会继续探索数据。

In [ ]:
figure_global(ds.isel(time=0, lead_time=3, ensemble=0).tcwv, *get_lat_lon("San Jose"))

完成此 Notebook 后，请关闭内核（在顶部菜单中，选择“Kernel”，然后选择“Shut Down Kernel”）。这将卸载 GPU 上的模型和所有张量，确保我们可以在接下来的练习中刷新开始。然后请继续前往[练习 2 的 Notebook](./exercise_02_validation.ipynb)，我们将更详细地检查预测。

## 额外内容

上面的例子使用了一种相对简单的球形高斯扰动。LBNL 和 NVIDIA 的研究人员开发了一种专门用于全球 AI 预报的扰动方法（见上面的出版物[第一部分](https://arxiv.org/abs/2408.03100)和[第二部分](https://arxiv.org/abs/2408.01581v1)）。他们的扰动方法以及跨多个独立训练的模型检查点的推理，对于首次科学合理地评估大型 AI 集成至关重要。如果您有额外的时间，可以按照下面的说明设置扰动方法（这里我们不会深入探讨跨多个检查点的推理）。

一个好的扰动方法的目标是为每个变量提供现实的不确定性测量。常见的一个方法是生成向量，它捕捉一个系统的不稳定性。快速变化的变量比慢变化的变量受到更大的扰动。扰动的强度通过比较无扰动预报和带有更简单的种子扰动的预报之间的差异来确定。因此，生成向量扰动在实际预报开始之前已经使用了我们的模型。下面所用的方法也对每个半球分别应用特定的重新缩放。

In [ ]:
skill_path = "./data/sfno_linear_skill_48h.nc"
noise_amplitude_seed = get_noise_vector(
    model,
    skill_path=skill_path,
    noise_amplification=0.35,
    vars=["z500"],
)
noise_amplitude_iter = get_noise_vector(
    model,
    skill_path=skill_path,
    noise_amplification=0.35,
)
seeding_perturbation = CorrelatedSphericalGaussian(noise_amplitude=noise_amplitude_seed)

hcbv = HemisphericCentredBredVector(
    model=model,
    data=ncar_era5,
    time=start_time,
    noise_amplitude=noise_amplitude_iter,
    integration_steps=3,
    seeding_perturbation_method=seeding_perturbation,
)

一旦设置了 `HemisphericCentredBredVector`，就可以简单地将其传递到与上面相同的 Earth2Studio 工作流中。

In [ ]:
io_extra = KVBackend()
io_extra = run.ensemble(
    [start_time],
    nsteps=nsteps,
    nensemble=nensemble,
    prognostic=model,
    data=ncar_era5,
    io=io_extra,
    perturbation=hcbv,
    batch_size=batch_size,
    output_coords=output_coords,
)
ds_extra = io_extra.to_xarray()

快速看一下结果。

In [ ]:
plot_global_ensemble(ds_extra.tcwv, start_time, *get_lat_lon("Tokyo"), cb_label="tcwv [kg/m²]", ilead_time=10)

请记得在进行下一个练习之前关闭您的内核。